![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Redis Online Feature Store with Feast

In this recipe, we will learn all about [Feature Stores](https://redis.io/solutions/feature-stores/) with **Redis** and **Feast**. This guide is an adaptation of the [Feast Tutorial](https://docs.feast.dev/tutorials/tutorials-overview/real-time-credit-scoring-on-aws) that uses [Redis as the online feature store](https://docs.feast.dev/reference/online-stores/redis).

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/feat/RAAE-447/feature-store-recipe/python-recipes/feature-store/01_feast_credit_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Environment Setup

### Install Python Dependencies

In [1]:
!pip install -q feast['redis']==0.42.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 11.8 MB/s eta 0:00:00


### Install Redis Stack

In this recipe, **Redis** will be used to store and fetch ML model features through Feast. **We need to make sure we have a Redis instance available.**

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [2]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.io/cloud/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker:

    ```bash
    docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest
    ```

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace `REDIS_PASSWORD`, `REDIS_HOST` and `REDIS_PORT` values with your own.

In [3]:
import os
import warnings

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

# Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# See https://docs.feast.dev/reference/online-stores/redis for details on Feast connection to Redis
REDIS_URL_FEAST = f"{REDIS_HOST}:{REDIS_PORT},ssl=false,password={REDIS_PASSWORD}"

## Load features dataset

Below we will make a `creditscore/` directory which will be the home of our Feast repo. We'll create and store additional files there down the road. For now we are loading dataset files into `creditscore/data`.

In [4]:
%%bash
mkdir creditscore
mkdir creditscore/data
wget https://github.com/antonum/feast-redis/raw/refs/heads/main/creditscore/data/credit_history.parquet -q -P creditscore/data
wget https://github.com/antonum/feast-redis/raw/refs/heads/main/creditscore/data/zipcode_table.parquet -q -P creditscore/data
wget https://github.com/antonum/feast-redis/raw/refs/heads/main/creditscore/data/loan_table.parquet -q -P creditscore/data

mkdir: cannot create directory ‘creditscore’: File exists
mkdir: cannot create directory ‘creditscore/data’: File exists


### Creating feature_store.yaml

`feature_store.yaml` is used to configure a feature store with Feast. The file must be located at the root of a feature repository `creditscore/`.

See [Redis | Feast Documentation](https://docs.feast.dev/reference/online-stores/redis) for the details of configuring Redis as an online store.

In [5]:
feature_store_config = \
f"""project: creditscore
registry: data/registry.db
provider: local
online_store:
    type: redis
    connection_string: {REDIS_URL_FEAST}
entity_key_serialization_version: 2
"""

with open('creditscore/feature_store.yaml', "w") as file:
    file.write(feature_store_config)


In [6]:
# Print our feature_store.yaml
! cat creditscore/feature_store.yaml

project: creditscore
registry: data/registry.db
provider: local
online_store:
    type: redis
    connection_string: localhost:6379,ssl=false,password=
entity_key_serialization_version: 2


### Feature Definitions

A feature repository can also contain one or more Python files that contain feature definitions.

In [7]:
%%writefile creditscore/features.py

from datetime import timedelta

from feast import (
    Entity,
    Field,
    FeatureView,
    ValueType,
    FileSource
  )
from feast.types import Float32, Int64, String


# Feature Definitions

## Zipcode Features
zipcode = Entity(
    name="zipcode",
    value_type=ValueType.STRING
)
zipcode_source = FileSource(
    path="data/zipcode_table.parquet",
    timestamp_field="event_timestamp",
    #event_timestamp_column="event_timestamp",
    created_timestamp_column="created_timestamp",
)
zipcode_features = FeatureView(
    name="zipcode_features",
    entities=[zipcode],
    ttl=timedelta(days=3650),
    schema=[
        Field(name="city", dtype=String),
        Field(name="state", dtype=String),
        Field(name="location_type", dtype=String),
        Field(name="tax_returns_filed", dtype=Int64),
        Field(name="population", dtype=Int64),
        Field(name="total_wages", dtype=Int64),
    ],
    source=zipcode_source,
)


## Credit History Features
dob_ssn = Entity(
    name="dob_ssn",
    description="Date of birth and last four digits of social security number",
    value_type=ValueType.STRING
)
credit_history_source = FileSource(
    path="data/credit_history.parquet",
    timestamp_field="event_timestamp",
    #event_timestamp_column="event_timestamp",
    created_timestamp_column="created_timestamp",

)
credit_history = FeatureView(
    name="credit_history",
    entities=[dob_ssn],
    ttl=timedelta(days=3650),
    schema=[
        Field(name="dob_ssn", dtype=String),  # Add entity column for dob_ssn
        Field(name="credit_card_due", dtype=Int64),
        Field(name="mortgage_due", dtype=Int64),
        Field(name="student_loan_due", dtype=Int64),
        Field(name="vehicle_loan_due", dtype=Int64),
        Field(name="hard_pulls", dtype=Int64),
        Field(name="missed_payments_2y", dtype=Int64),
        Field(name="missed_payments_1y", dtype=Int64),
        Field(name="missed_payments_6m", dtype=Int64),
        Field(name="bankruptcies", dtype=Int64),
    ],
    source=credit_history_source,
)

Overwriting creditscore/features.py


### Create Feast repository

In [8]:
%cd creditscore/
!feast apply

/content/creditscore
No project found in the repository. Using project name creditscore defined in feature_store.yaml
Applying changes for project creditscore
Deploying infrastructure for credit_history
Deploying infrastructure for zipcode_features


### Materialize features into Redis

Load data from feature views (parquet files) into the online store (Redis). Use `feast materialize-incremental` to update online store with changes since the last `materialize` call.

In [9]:
warnings.simplefilter("ignore", DeprecationWarning)

!feast materialize-incremental 2025-01-24T16:57:10
%cd ..

Materializing 2 feature views to 2025-01-24 16:57:10+00:00 into the redis online store.

credit_history from 2025-01-24 16:57:10+00:00 to 2025-01-24 16:57:10+00:00:
0it [00:00, ?it/s]
zipcode_features from 2025-01-24 16:57:10+00:00 to 2025-01-24 16:57:10+00:00:
0it [00:00, ?it/s]
/content


## Retreive feature vector from the Redis Online Store

`feast apply` and `feast materialize` initialized our feature store, so now we can request features from the Redis online store with `store.get_online_features()` call.

In [10]:
from feast import FeatureStore
warnings.simplefilter("ignore", DeprecationWarning)


store = FeatureStore(repo_path="creditscore/")
feast_features = [
        "zipcode_features:city",
        "zipcode_features:state",
        "zipcode_features:location_type",
        "zipcode_features:tax_returns_filed",
        "zipcode_features:population",
        "zipcode_features:total_wages",
        "credit_history:credit_card_due",
        "credit_history:mortgage_due",
        "credit_history:student_loan_due",
        "credit_history:vehicle_loan_due",
        "credit_history:hard_pulls",
        "credit_history:missed_payments_2y",
        "credit_history:missed_payments_1y",
        "credit_history:missed_payments_6m",
        "credit_history:bankruptcies",
    ]
zipcode = "76104"
dob_ssn = "19630621_4278"

feature_vector = store.get_online_features(
    features = feast_features,
    entity_rows = [{"zipcode": zipcode, "dob_ssn": dob_ssn}]
)
feature_vector.to_dict()

{'zipcode': ['76104'],
 'dob_ssn': ['19630621_4278'],
 'tax_returns_filed': [6058],
 'state': ['TX'],
 'total_wages': [142325465],
 'city': ['FORT WORTH'],
 'population': [10534],
 'location_type': ['PRIMARY'],
 'hard_pulls': [1],
 'missed_payments_2y': [0],
 'missed_payments_1y': [0],
 'student_loan_due': [44375],
 'vehicle_loan_due': [11506],
 'credit_card_due': [3343],
 'bankruptcies': [0],
 'mortgage_due': [378847],
 'missed_payments_6m': [0]}

## Examine source data

`credit_history.parquet` and `zipcode_table.parquet` contains data that would be exposed by our featurestoe as both online and offline features.  `loan_table.parquet` is used only to train the model and contains historical loan request submissions and target value as approve/deny in `loan_status`.

In [11]:
import pandas as pd

pd.read_parquet("creditscore/data/credit_history.parquet")

# zipcode_table.parquet
# loan_table.parquet

,event_timestamp,dob_ssn,credit_card_due,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies,created_timestamp
0,2020-04-26 18:01:04.746575,19530219_5179,8419,91803,22328,15078,0,1,0,0,0,2020-04-26 18:01:04.746575
1,2020-04-26 18:01:04.746575,19781116_7723,2944,741165,2515,28605,0,3,3,1,0,2020-04-26 18:01:04.746575
2,2020-04-26 18:01:04.746575,19931128_5771,833,976522,33000,21733,9,7,0,0,0,2020-04-26 18:01:04.746575
3,2020-04-26 18:01:04.746575,19500806_6783,5936,1553523,48955,26219,1,0,0,0,0,2020-04-26 18:01:04.746575
4,2020-04-26 18:01:04.746575,19620322_7692,1575,1067381,9501,15814,1,1,0,0,0,2020-04-26 18:01:04.746575
...,...,...,...,...,...,...,...,...,...,...,...,...
2033293,2021-08-29 18:01:04.746575,19621030_8837,9045,1106144,25760,13826,8,5,2,1,0,2021-08-29 18:01:04.746575
2033294,2021-08-29 18:01:04.746575,19810914_5886,5065,1376873,20594,13948,8,5,1,1,0,2021-08-29 18:01:04.746575
2033295,2021-08-29 18:01:04.746575,19491025_8061,738,273532,24113,15902,10,1,2,1,0,2021-08-29 18:01:04.746575
2033296,2021-08-29 18:01:04.746575,19751125_4615,3443,1534792,43133,16294,4,6,2,1,0,2021-08-29 18:01:04.746575


## Machine Learning Model Training

While our feature store at this point already complete, let's put it to a good use and introduce a `LoadRequestModel` that we will train, using `get_historical_features()` and use to make predictions with `get_online_features()`

In [12]:
from pathlib import Path

import feast
import joblib
import pandas as pd

from sklearn import tree
from sklearn.exceptions import NotFittedError
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils.validation import check_is_fitted
warnings.simplefilter("ignore", DeprecationWarning)


class LoadRequestModel:
    """
    ML model to classify whether a person should
    get approved or rejected for a loan based on a variety of
    input factors.
    """
    categorical_features = [
        "person_home_ownership",
        "loan_intent",
        "city",
        "state",
        "location_type",
    ]

    feast_features = [
        "zipcode_features:city",
        "zipcode_features:state",
        "zipcode_features:location_type",
        "zipcode_features:tax_returns_filed",
        "zipcode_features:population",
        "zipcode_features:total_wages",
        "credit_history:credit_card_due",
        "credit_history:mortgage_due",
        "credit_history:student_loan_due",
        "credit_history:vehicle_loan_due",
        "credit_history:hard_pulls",
        "credit_history:missed_payments_2y",
        "credit_history:missed_payments_1y",
        "credit_history:missed_payments_6m",
        "credit_history:bankruptcies",
    ]

    target = "loan_status"
    model_filename = "model.bin"
    encoder_filename = "encoder.bin"

    def __init__(self,secret=""):
        # Load model
        if Path(self.model_filename).exists():
            self.classifier = joblib.load(self.model_filename)
        else:
            self.classifier = tree.DecisionTreeClassifier()

        # Load ordinal encoder
        if Path(self.encoder_filename).exists():
            self.encoder = joblib.load(self.encoder_filename)
        else:
            self.encoder = OrdinalEncoder()

        # Set up feature store
        self.fs = feast.FeatureStore(repo_path="creditscore/")
        #if secret and (":" in secret):
        #    self.fs.config.online_store.connection_string=secret

    def train(self, loans):
        train_X, train_Y = self._get_training_features(loans)

        self.classifier.fit(train_X[sorted(train_X)], train_Y)
        joblib.dump(self.classifier, self.model_filename)

    def _get_training_features(self, loans):
        training_df = self.fs.get_historical_features(
            entity_df=loans, features=self.feast_features
        ).to_df()

        self._fit_ordinal_encoder(training_df)
        self._apply_ordinal_encoding(training_df)
        #print(training_df.head())
        train_X = training_df[
            training_df.columns.drop(self.target)
            .drop("event_timestamp")
            .drop("created_timestamp__")
            .drop("loan_id")
            .drop("zipcode")
            .drop("dob_ssn")
        ]
        train_X = train_X.reindex(sorted(train_X.columns), axis=1)
        train_Y = training_df.loc[:, self.target]

        return train_X, train_Y

    def _fit_ordinal_encoder(self, requests):
        self.encoder.fit(requests[self.categorical_features])
        joblib.dump(self.encoder, self.encoder_filename)

    def _apply_ordinal_encoding(self, requests):
        requests[self.categorical_features] = self.encoder.transform(
            requests[self.categorical_features]
        )

    def predict(self, request):
        # Get online features from Feast
        feature_vector = self._get_online_features_from_feast(request)

        # Join features to request features
        features = request.copy()
        features.update(feature_vector)
        features_df = pd.DataFrame.from_dict(features)

        # Apply ordinal encoding to categorical features
        self._apply_ordinal_encoding(features_df)

        # Sort columns
        features_df = features_df.reindex(sorted(features_df.columns), axis=1)

        # Drop unnecessary columns
        features_df = features_df[features_df.columns.drop("zipcode").drop("dob_ssn")]

        # Make prediction
        features_df["prediction"] = self.classifier.predict(features_df)

        # return result of credit scoring
        return features_df["prediction"].iloc[0]

    def _get_online_features_from_feast(self, request):
        zipcode = request["zipcode"][0]
        dob_ssn = request["dob_ssn"][0]

        return self.fs.get_online_features(
            entity_rows=[{"zipcode": zipcode, "dob_ssn": dob_ssn}],
            features=self.feast_features,
        ).to_dict()

    def is_model_trained(self):
        try:
            check_is_fitted(self.classifier, "tree_")
        except NotFittedError:
            return False
        return True


## Initialize the model

Now we need to train the model and make a sample prediction. After training is completed you'll see `model.bin` and `encoder.bin` files in the filesystem.

In [13]:
# Create model
model = LoadRequestModel()

# Train model (using Parquet for zipcode and credit history features)
if not model.is_model_trained():
    print("Model not trained. Performing training.")
    # Get historic loan data
    loans = pd.read_parquet("creditscore/data/loan_table.parquet")
    model.train(loans)

### Make a Loan Request

We will now use our trained ML model and feature store to predict whether or not you would get a loan.

While making a loan request, make sure that `dob_ssn` and `zipcode` values do exist in the source datasets. You can examine source datasets with `pd.read_parquet("creditscore/data/credit_history.parquet")`

In [14]:
import ipywidgets as widgets

# initialize loan request with sample data
loan_request = {
    "zipcode": [76104],
    "dob_ssn": ["19630621_4278"],
    "person_age": [63],
    "person_income": [159000],
    "person_home_ownership": ["RENT"],
    "person_emp_length": [123.0],
    "loan_intent": ["PERSONAL"],
    "loan_amnt": [5000],
    "loan_int_rate": [16.02],
}


slider_income = widgets.IntSlider(loan_request["person_income"][0], max=1000000, min=0, description="Income: ")
slider_amount = widgets.IntSlider(loan_request["loan_amnt"][0], max=1000000, min=0, description="Loan Amount: ")
slider_int_rate = widgets.IntSlider(loan_request["loan_int_rate"][0], max=90, min=1, description="Interest Rate: ")

print("Select amounts below:")
display(slider_income, slider_amount, slider_int_rate)


Select amounts below:


IntSlider(value=159000, description='Income: ', max=1000000)

IntSlider(value=5000, description='Loan Amount: ', max=1000000)

IntSlider(value=16, description='Interest Rate: ', max=90, min=1)

In [15]:
loan_request["person_income"] = [slider_income.value]
loan_request["loan_amnt"] = [slider_amount.value]
loan_request["loan_int_rate"] = [slider_int_rate.value]


# Make online prediction (using Redis for retrieving online features)
result = model.predict(loan_request)

if result == 0:
    print("Loan approved!")
elif result == 1:
    print("Loan rejected!")

warnings.simplefilter("ignore", DeprecationWarning)

Loan rejected!


Let's inspect an individual loan request payload.

In [16]:
warnings.simplefilter("ignore", DeprecationWarning)

loan_request_df = pd.DataFrame.from_dict(loan_request)
loan_request_df.transpose()

,0
zipcode,76104
dob_ssn,19630621_4278
person_age,63
person_income,159000
person_home_ownership,RENT
person_emp_length,123.0
loan_intent,PERSONAL
loan_amnt,5000
loan_int_rate,16


Let's inspect the feature store features pulled from Redis.

In [17]:
warnings.simplefilter("ignore", DeprecationWarning)

feature_vector = model._get_online_features_from_feast(loan_request)
feature_vector_df=pd.DataFrame.from_dict(feature_vector)
feature_vector_df.transpose()

,0
zipcode,76104
dob_ssn,19630621_4278
tax_returns_filed,6058
state,TX
total_wages,142325465
city,FORT WORTH
population,10534
location_type,PRIMARY
hard_pulls,1
missed_payments_2y,0


Join the features to see the entire input sent to the credit prediction model.

In [18]:
features = loan_request.copy()
features.update(feature_vector)
features_df = pd.DataFrame.from_dict(features)
features_df.transpose()

,0
zipcode,76104
dob_ssn,19630621_4278
person_age,63
person_income,159000
person_home_ownership,RENT
person_emp_length,123.0
loan_intent,PERSONAL
loan_amnt,5000
loan_int_rate,16
tax_returns_filed,6058


In [19]:
result = model.predict(loan_request)

if result == 0:
    print("Loan approved!")
elif result == 1:
    print("Loan rejected!")

Loan rejected!


## Benchmarking

The key advantage of Redis as a Online feature store is it's ability to very quickly retreive features on request. Below, we'll retreive the same data from Online store (Redis) and from the Offline store (parquet) and measure execution time.

In [20]:
store = FeatureStore(repo_path="creditscore/")
feast_features = [
        "zipcode_features:city",
        "zipcode_features:state",
        "zipcode_features:location_type",
        "zipcode_features:tax_returns_filed",
        "zipcode_features:population",
        "zipcode_features:total_wages",
        "credit_history:credit_card_due",
        "credit_history:mortgage_due",
        "credit_history:student_loan_due",
        "credit_history:vehicle_loan_due",
        "credit_history:hard_pulls",
        "credit_history:missed_payments_2y",
        "credit_history:missed_payments_1y",
        "credit_history:missed_payments_6m",
        "credit_history:bankruptcies",
    ]
zipcode = "76104"
dob_ssn = "19630621_4278"

entity_rows=[{"zipcode": zipcode, "dob_ssn": dob_ssn}]
entity_rows_df=pd.DataFrame(entity_rows)
entity_rows_df["event_timestamp"]=pd.to_datetime("2020-04-26 18:01:04.746575")
entity_rows_df['zipcode'] = entity_rows_df['zipcode'].astype(int)

Online feature store retrieval benchmark:

In [21]:
%%timeit

online_features = store.get_online_features(
    features = feast_features,
    entity_rows = entity_rows
)

15.5 ms ± 772 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Offline feature store retrieval benchmark:

In [22]:
%%timeit

offline_features= store.get_historical_features(
    entity_df = entity_rows_df,
    features = feast_features
).to_df()

3.77 s ± 449 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


>Note: That's more than a 100x difference. (typically)

In [23]:
# retreive sample of keys from redis
from redis import Redis

redis_client = Redis.from_url(REDIS_URL)
redis_client.keys()

[b'\x02\x00\x00\x00zipcode\x04\x00\x00\x00\x08\x00\x00\x00y\x14\x01\x00\x00\x00\x00\x00creditscore',
 b'\x02\x00\x00\x00dob_ssn\x02\x00\x00\x00\r\x00\x00\x0019770705_9126creditscore',
 b'\x02\x00\x00\x00dob_ssn\x02\x00\x00\x00\r\x00\x00\x0019991125_5865creditscore',
 b'\x02\x00\x00\x00dob_ssn\x02\x00\x00\x00\r\x00\x00\x0019861118_3193creditscore',
 b'\x02\x00\x00\x00dob_ssn\x02\x00\x00\x00\r\x00\x00\x0020011102_9929creditscore',
 b'\x02\x00\x00\x00zipcode\x04\x00\x00\x00\x08\x00\x00\x00(G\x00\x00\x00\x00\x00\x00creditscore',
 b'\x02\x00\x00\x00dob_ssn\x02\x00\x00\x00\r\x00\x00\x0019860122_3218creditscore',
 b'\x02\x00\x00\x00dob_ssn\x02\x00\x00\x00\r\x00\x00\x0019991027_1238creditscore',
 b'\x02\x00\x00\x00zipcode\x04\x00\x00\x00\x08\x00\x00\x00\x0f\xb9\x00\x00\x00\x00\x00\x00creditscore',
 b'\x02\x00\x00\x00zipcode\x02\x00\x00\x00\x05\x00\x00\x0070744creditscore',
 b'\x02\x00\x00\x00zipcode\x04\x00\x00\x00\x08\x00\x00\x00\x01%\x01\x00\x00\x00\x00\x00creditscore',
 b'\x02\x00\x00\x00zi

### Cleanup

In [24]:
# %cd creditscore/
# !feast teardown
# %cd ..